## load our dependencies

In [94]:
import pandas as pd
import re

# load our dataframe

In [95]:
audible_path = './audible_uncleaned.csv'
audible_df = pd.read_csv(audible_path)

print("Audible Info")
print("*" * 100)
print(audible_df.info())
print("*" * 100)

Audible Info
****************************************************************************************************
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87489 entries, 0 to 87488
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   name         87489 non-null  object
 1   author       87489 non-null  object
 2   narrator     87489 non-null  object
 3   time         87489 non-null  object
 4   releasedate  87489 non-null  object
 5   language     87489 non-null  object
 6   stars        87489 non-null  object
 7   price        87489 non-null  object
dtypes: object(8)
memory usage: 5.3+ MB
None
****************************************************************************************************


# extract info from 'stars'

##### '**4.5 out of 5 stars41 ratings**' ➡️ stars: 4.5 | ratings: 41 | rated: yes

##### '**Not rated yet**' ➡️ stars: NaN | ratings: NaN | rated: no 

In [96]:
# Rename Stars to use it with correct float values
audible_df.rename(columns={'stars': 'original_stars'}, inplace=True)

# Step 1: Extract star ratings as float
audible_df['stars'] = audible_df['original_stars'].str.extract(r'(\d+\.?\d*)').astype(float)

# Step 2: Extract rating counts as integers
audible_df['ratings'] = audible_df['original_stars'].str.extract(r'(\d+) ratings').astype(float)

# Step 3: Add 'rated' field
audible_df['rated'] = audible_df['original_stars'].apply(lambda x: 'no' if 'Not rated yet' in x else 'yes')

# Drop the original 'stars' column if it's no longer needed
audible_df.drop(columns=['original_stars'], inplace=True)

In [106]:
print("*" * 100)
print(audible_df[['stars', 'ratings', 'rated']].head())
print("*" * 100)

****************************************************************************************************
   stars  ratings rated
0    5.0     34.0   yes
1    4.5     41.0   yes
2    4.5     38.0   yes
3    4.5     12.0   yes
4    4.5    181.0   yes
****************************************************************************************************


# split 'releasedate' into more attributes

##### '**04-08-08**' ➡️ month: 04 | day: 08 | year: 08

In [98]:
# Split the 'releasedate' column into day, month, and year
audible_df[['day', 'month', 'year']] = audible_df['releasedate'].str.split('-', expand=True)

# Convert the new columns to integers
audible_df['day'] = audible_df['day'].apply(lambda x: x.zfill(2))
audible_df['month'] = audible_df['month'].apply(lambda x: x.zfill(2))
# Format the year as a four-digit year
audible_df['year'] = audible_df['year'].apply(lambda x: int(x) + 2000 if int(x) < 100 else int(x))

In [107]:
print("*" * 100)
print(audible_df[['day', 'month', 'year']].head())
print("*" * 100)

****************************************************************************************************
  day month  year
0  04    08  2008
1  01    05  2018
2  06    11  2020
3  05    10  2021
4  13    01  2010
****************************************************************************************************


# fix 'author' and 'narrator' to exclude _______by:

****some books may have multiple authors****

##### 'Writtenby:GeronimoStilton' ➡️ author: Geronimo Stilton 
##### 'Writtenby:RonaldL.Smith' ➡️ author: Ronald L. Smith 
##### 'Writtenby:HumzaArshad,HenryWhite' ➡️ author: Humza Arshad, Henry White 

##### 'Narratedby:BillLobely' ➡️ narrator: Bill Lobely 

In [100]:
# Remove 'Writtenby:' prefix and add spaces between names for 'author' field
audible_df['author'] = audible_df['author'].str.replace('Writtenby:', '', regex=False)
audible_df['author'] = audible_df['author'].apply(lambda x: re.sub(r'(\w)([A-Z])', r'\1 \2', ', '.join(x.split(','))))

# Remove 'Narratedby:' prefix and add spaces between names for 'narrator' field
audible_df['narrator'] = audible_df['narrator'].str.replace('Narratedby:', '', regex=False)
audible_df['narrator'] = audible_df['narrator'].apply(lambda x: re.sub(r'(\w)([A-Z])', r'\1 \2', x))


In [110]:
print("*" * 100)
print(audible_df[['author', 'narrator']].head())
print("*" * 100)

****************************************************************************************************
             author         narrator
0  Geronimo Stilton      Bill Lobely
1      Rick Riordan   Robbie Daymond
2       Jeff Kinney      Dan Russell
3      Rick Riordan  Soneela Nankani
4      Rick Riordan  Jesse Bernstein
****************************************************************************************************


# convert time into minutes for better use of units

##### '**time:11 hrs and 16 mins**' ➡️  minutes: 676

In [115]:
# Use lambda function to convert time to minutes
audible_df['minutes'] = audible_df['time'].apply(
    lambda x: (
        (int(re.search(r'(\d+)\s*hr', x).group(1)) * 60 if re.search(r'(\d+)\s*hr', x) else 0) +
        (int(re.search(r'(\d+)\s*min', x).group(1)) if re.search(r'(\d+)\s*min', x) else 0)
    )
)

In [116]:
print("*" * 100)
print(audible_df[['minutes']].head())
print("*" * 100)

****************************************************************************************************
   minutes
0      140
1      788
2      123
3      676
4      600
****************************************************************************************************


In [117]:
print("Final Table")
audible_df = audible_df[['name','author','narrator','time','minutes','releasedate','month', 'day', 'year','language','stars','rated','ratings','price']]
audible_df.to_csv('audible_output.csv', index=False)
print(audible_df.head())

Final Table
                                         name            author  \
0                  Geronimo Stilton #11 & #12  Geronimo Stilton   
1                            The Burning Maze      Rick Riordan   
2                                The Deep End       Jeff Kinney   
3                        Daughter of the Deep      Rick Riordan   
4  The Lightning Thief: Percy Jackson, Book 1      Rick Riordan   

          narrator                time  minutes releasedate month day  year  \
0      Bill Lobely   2 hrs and 20 mins      140    04-08-08    08  04  2008   
1   Robbie Daymond   13 hrs and 8 mins      788    01-05-18    05  01  2018   
2      Dan Russell    2 hrs and 3 mins      123    06-11-20    11  06  2020   
3  Soneela Nankani  11 hrs and 16 mins      676    05-10-21    10  05  2021   
4  Jesse Bernstein              10 hrs      600    13-01-10    01  13  2010   

  language  stars rated  ratings   price  
0  English    5.0   yes     34.0  468.00  
1  English    4.5   yes 